In [2]:
###  5.3.1 标准化
#也称去均值和方差按比例缩放

#数据集的 标准化 对scikit-learn中实现的大多数机器学习算法来说是 常见的要求 。如果个别特征或多或少看起来不是很像标准正态分布(具有零均值和单位方差)，那么它们的表现力可能会较差。
#在实际情况中,我们经常忽略特征的分布形状，直接经过去均值来对某个特征进行中心化，再通过除以非常量特征(non-constant features)的标准差进行缩放。


## StandardScaler
from sklearn import preprocessing
import numpy as np
X_train = np.array([[ 1., -1.,  2.],
                    [ 2.,  0.,  0.],
                    [ 0.,  1., -1.]])
scaler = preprocessing.StandardScaler().fit(X_train)
scaler


StandardScaler()

In [6]:

# 均值
scaler.mean_



array([1.        , 0.        , 0.33333333])

In [11]:
#标准方差
scaler.scale_


array([0.81649658, 0.81649658, 1.24721913])

In [5]:
X_scaled = scaler.transform(X_train)
X_scaled

array([[ 0.        , -1.22474487,  1.33630621],
       [ 1.22474487,  0.        , -0.26726124],
       [-1.22474487,  1.22474487, -1.06904497]])

In [9]:
## 经过缩放后的数据具有零均值以及标准方差
#均值
X_scaled.mean(axis=0)

array([0., 0., 0.])

In [10]:
#方差
X_scaled.std(axis=0)

array([1., 1., 1.])

In [12]:
## 5.3.1.1  将特征缩放至特定范围内 
#一种标准化是将特征缩放到给定的最小值和最大值之间，通常在零和一之间，或者也可以将每个特征的最大绝对值转换至单位大小。可以分别使用 MinMaxScaler 和 MaxAbsScaler 实现。

#以下是一个将简单的数据矩阵缩放到[0, 1]的例子:
X_train = np.array([[ 1., -1.,  2.],
                    [ 2.,  0.,  0.],
                    [ 0.,  1., -1.]])
min_max_scaler = preprocessing.MinMaxScaler()
X_train_minmax = min_max_scaler.fit_transform(X_train)
X_train_minmax



array([[0.5       , 0.        , 1.        ],
       [1.        , 0.5       , 0.33333333],
       [0.        , 1.        , 0.        ]])

In [13]:
# 同样的转换实例可以被用与在训练过程中不可见的测试数据:实现和训练数据一致的缩放和移位操作:
X_test = np.array([[-3., -1.,  4.]])
X_test_minmax = min_max_scaler.transform(X_test)   #对测试数据进行标准化（注意此处的4在训练集中未出现）
X_test_minmax


array([[-1.5       ,  0.        ,  1.66666667]])

In [15]:
### 5.3.3 归一化
#通过对原始数据进行线性变换把数据映射到[0,1]或[-1,1]之间，常用的变换函数最大最小值归一化

X = [[ 1., -1.,  2.],
     [ 2.,  0.,  0.],
     [ 0.,  1., -1.]]
X_normalized = preprocessing.normalize(X, norm='l2') #参数norm是用于标准化每个非0数据的范式，可用的值是 l1和l2 范式，默认值是l2范式。

X_normalized

array([[ 0.40824829, -0.40824829,  0.81649658],
       [ 1.        ,  0.        ,  0.        ],
       [ 0.        ,  0.70710678, -0.70710678]])

In [16]:
#preprocessing 预处理模块提供的 Normalizer 工具类使用 Transformer API 实现了相同的操作(即使在这种情况下，fit方法是无用的：该类是无状态的，因为该操作独立处理样本).
normalizer = preprocessing.Normalizer().fit(X)  # fit does nothing
normalizer

Normalizer()

In [17]:
#在这之后归一化实例可以被使用在样本向量中，像任何其他转换器一样:
normalizer.transform(X)   #即，将训练和预测分开，可应用到不同数据集。
normalizer.transform([[-1.,  1., 0.]])

array([[-0.70710678,  0.70710678,  0.        ]])

In [19]:
### 5.3.4 类别特征编码
#在机器学习中，特征经常不是连续的数值型的而是标称型的(categorical)。
#这些特征能够被有效地编码成整数，比如 ["male", "from US", "uses Internet Explorer"] 可以被表示为 [0, 1, 3]。

#要把标称型特征(categorical features) 转换为这样的整数编码(integer codes), 我们可以使用 OrdinalEncoder 。 
#这个估计器把每一个categorical feature变换成 一个新的整数数字特征 (0 到 n_categories - 1):
enc = preprocessing.OrdinalEncoder()
X = [['male', 'from US', 'uses Safari'], ['female', 'from Europe', 'uses Firefox']]
enc.fit(X)
enc.transform([['female', 'from US', 'uses Safari']]) #这样的整数特征表示并不能在scikit-learn的估计器中直接使用，因为这样的连续输入，
                                                      #估计器会认为类别之间是有序的，但实际却是无序的。


array([[0., 1., 1.]])

In [21]:
## one-hot编码
#另外一种将标称型特征转换为能够被scikit-learn中模型使用的编码是one-of-K， 又称为 独热码或dummy encoding。 
#这种编码类型已经在类OneHotEncoder中实现。该类把每一个具有n_categories个可能取值的categorical特征变换为
#长度为n_categories的二进制特征向量，里面只有一个地方是1，其余位置都是0。

enc = preprocessing.OneHotEncoder()
X = [['male', 'from US', 'uses Safari'], ['female', 'from Europe', 'uses Firefox']]
enc.fit(X)
enc.transform([['female', 'from US', 'uses Safari'],
               ['male', 'from Europe', 'uses Safari']]).toarray()

array([[1., 0., 0., 1., 0., 1.],
       [0., 1., 1., 0., 0., 1.]])

In [24]:
#如果训练数据可能缺少分类特性，通常最好指定handle_unknown='ignore'，而不是手动指定，虽然提供了这种功能 - categories参数。
enc = preprocessing.OneHotEncoder(handle_unknown='infrequent_if_exist')  #如果没有这个参数，且transform输入新类型时会报错。
X = [['male', 'from US', 'uses Safari'], ['female', 'from Europe', 'uses Firefox']]
enc.fit(X)
enc.transform([['female', 'from Asia', 'uses Chrome']]).toarray()  #'from Asia'和'uses Chrome' 被分配到默认位置

array([[1., 0., 0., 0., 0., 0.]])

In [32]:
### 5.3.5 离散化
#离散化 (Discretization) (有些时候叫 量化(quantization) 或 装箱(binning)) 提供了将连续特征划分为离散特征值的方法。 
#某些具有连续特征的数据集会受益于离散化，因为离散化可以把具有连续属性的数据集变换成只有名义属性(nominal attributes)的数据集。

## 5.3.5.1 K-bins 离散化
#KBinsDiscretizer 类使用k个等宽的bins把特征离散化。
X = np.array([[ -3., 5., 15 ],
              [  0., 6., 14 ],
              [  6., 3., 11 ]])
est = preprocessing.KBinsDiscretizer(n_bins=[3, 2, 2], encode='ordinal').fit(X) #默认情况下，输出是被 one-hot 编码到一个稀疏矩阵.此处是数字编码。
                                                                        #n_bins参数可以是数字-每个特征均分成k个箱子，也可以是list为每个特征指定箱子数。

#基于这些 bin 区间, X 就被变换成下面这样:
est.transform(X)  

array([[0., 1., 1.],
       [1., 1., 1.],
       [2., 0., 0.]])

In [33]:
## 5.3.5.2 Binarizer特征二值化
# class sklearn.preprocessing.Binarizer(*, threshold=0.0, copy=True)[source]¶
#根据阈值对数据进行二值化（将要素值设置为 0 或 1）。
#大于阈值的值映射到 1，而小于 或等于阈值映射到 0。默认阈值为 0， 只有正值映射到 1。

X = [[ 1., -1.,  2.],
     [ 2.,  0.,  0.],
     [ 0.,  1., -1.]]

binarizer = preprocessing.Binarizer().fit(X)  # fit does nothing
binarizer
binarizer.transform(X)  #大于阈值0的被映射为1，否则被映射为0

array([[1., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.]])